In [100]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import xgboost as xgb
import time
import itertools
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler


In [101]:
# df_TagDesc = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\CSV\Not for Processing\TagDesc.csv')

df_All_1 = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\1 Preprocess\Merge Data\contData_all_Avg_1o2.csv')
df_All_2 = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\1 Preprocess\Merge Data\contData_all_Avg_2o2.csv')
# Concatenate (union) the dataframes
df_All = pd.concat([df_All_1, df_All_2], ignore_index=True)

print(df_All.head())

   425_pct_Al  M_Value  C4_pct_Eth  C4_pct_H2O  C4_pct_Hex  \
0    6.307630  3.50893    2.508420    21.96340    0.659256   
1    6.308335  3.50863    2.511135    21.96865    0.659930   
2    6.309390  3.50818    2.515210    21.97655    0.660942   
3    6.310090  3.50788    2.517925    21.98185    0.661616   
4    6.310790  3.50758    2.520640    21.98715    0.662291   

   HydWtr_pct_Ammonia  HydWtr_Na2O   DI55152   FC55003       FC55552  \
0            0.908765     2.712180  0.925255  4919.290  41564.100000   
1            0.907643     2.805415  0.924281  5039.445  41558.250000   
2            0.905959     2.945270  0.923773  5268.620  41524.066667   
3            0.904837     3.038505  0.924103  5349.320  41500.275000   
4            0.903715     3.131740  0.924258  5563.165  41466.625000   

       FC55569  FFC55553  FFC55555    LC55555   PI55004   TC55552     TC55555  \
0  6918.110000  0.996975  0.751501  59.968800  1.306310  177.1880  180.125000   
1  6919.960000  0.998847  0.7505

In [102]:
# Set max columns to display
pd.set_option('display.max_columns', None)

In [103]:
# List of columns to exclude to run XGboost feature selection
exclude_columns = [#'Date',
                 'C4_pct_Hex', 'HydWtr_Na2O',
                 'TC55555'  
                   ]

# Create a new DataFrame without the excluded columnsd
df_All = df_All.drop(columns=exclude_columns)

In [104]:
print(df_All.describe())

         425_pct_Al       M_Value    C4_pct_Eth    C4_pct_H2O  \
count  54701.000000  54701.000000  54701.000000  54701.000000   
mean       6.112584      3.604726      1.274754     20.756146   
std        0.316017      0.173736      0.677477      2.859486   
min        0.252306      1.461875      0.086948      7.932310   
25%        5.987750      3.506780      0.675285     18.331550   
50%        6.130470      3.595655      1.291115     20.731850   
75%        6.270075      3.696090      1.719630     22.989150   
max        8.020685      5.827565      8.711360     33.731050   

       HydWtr_pct_Ammonia       DI55152       FC55003       FC55552  \
count        54701.000000  54701.000000  54701.000000  54701.000000   
mean             0.969410      0.944165   5946.323971  36128.859598   
std              0.144668      0.042737    794.065219   4270.390546   
min              0.390025      0.815428   2991.532500  18800.350000   
25%              0.877716      0.915431   5446.200000  3499

In [105]:
# Splitting into train and test
X = df_All.drop('Butanol', axis=1)  # Assuming 'target' is your target column
y = df_All['Butanol']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# Define the parameter grid

param_grid = {
    'learning_rate': [0.05],  # Introduced some intermediate and higher values. 0.05, 0.06, 0.07, 0.08
    'n_estimators': [800],     # Expanded around 500, as it was your best result.
    'max_depth': [10],                # Introduced values around 7, which was your best depth. 6, 7, 8, 
    'subsample': [0.6],        # Expanded a bit to include values around 0.8. 0.8, 0.9, 1.0, 0.7, 0.6, 0.5, 0.4
    'colsample_bytree': [1],      # Values around your best result of 0.7.  0.6, 0.7, 0.8, 0.9, 1
    'gamma': [0],                   # Regularization parameter. Start with these values. 0, 0.1, 0.5 
    'alpha': [1.5],                   # L1 regularization term. 0, 0.1, 0.5, 0.7, 0.9, 1, 1.3, 1.4 1.5, 1.6, 2
    'lambda': [1.5],                    # L2 regularization term.  1, 1.5, 2
    'colsample_bylevel': [0.8],     # Additional column sampling.  0.6, 0.7,0.8 
    'colsample_bynode': [0.7]       # More granular column sampling  .0.6, 0.8
}

In [106]:
# param_grid = {
#     'learning_rate': [0.01, 0.1],
#     'n_estimators': [100, 500],
#     'max_depth': [3, 5, 7],
#     'subsample': [0.8, 1.0],
#     'colsample_bytree': [0.3, 0.5, 0.7]
# }
# Best Parameters: {'learning_rate': 0.1, 'n_estimators': 500, 'max_depth': 7, 'subsample': 0.8, 'colsample_bytree': 0.7}

xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', use_label_encoder=False)

def product_dict(**kwargs):
    keys = kwargs.keys()
    vals = kwargs.values()
    for instance in itertools.product(*vals):
        yield dict(zip(keys, instance))

best_score = float('inf')
best_params = None
results = []

# Loop over all combinations of hyperparameters
for params in product_dict(**param_grid):
    start_time = time.time()
    
    xgb_reg.set_params(**params)
    y_pred = cross_val_predict(xgb_reg, X_train, y_train, cv=3)

    mse = mean_squared_error(y_train, y_pred)
    mae = mean_absolute_error(y_train, y_pred)
    r2 = r2_score(y_train, y_pred)
    adj_r2 = 1 - (1-r2)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
    rmse = np.sqrt(mse)

    end_time = time.time()
    duration = end_time - start_time

    print(f"Evaluating {params} took {duration:.2f} seconds")
    print(f"MSE: {mse:.4f}, MAE: {mae:.4f}, R^2: {r2:.4f}, Adjusted R^2: {adj_r2:.4f}, RMSE: {rmse:.4f}")

    results.append({
        'Parameters': params,
        'MSE': mse,
        'MAE': mae,
        'R^2': r2,
        'Adjusted R^2': adj_r2,
        'RMSE': rmse,
        'Duration': duration
    })

    if mse < best_score:
        best_score = mse
        best_params = params

results_df = pd.DataFrame(results)

print("Best Parameters:", best_params)


Evaluating {'learning_rate': 0.05, 'n_estimators': 800, 'max_depth': 10, 'subsample': 0.6, 'colsample_bytree': 1, 'gamma': 0, 'alpha': 1.5, 'lambda': 1, 'colsample_bylevel': 0.6, 'colsample_bynode': 0.6} took 30.96 seconds
MSE: 4.9128, MAE: 1.1147, R^2: 0.9435, Adjusted R^2: 0.9435, RMSE: 2.2165
Evaluating {'learning_rate': 0.05, 'n_estimators': 800, 'max_depth': 10, 'subsample': 0.6, 'colsample_bytree': 1, 'gamma': 0, 'alpha': 1.5, 'lambda': 1, 'colsample_bylevel': 0.6, 'colsample_bynode': 0.7} took 37.21 seconds
MSE: 4.9213, MAE: 1.1115, R^2: 0.9434, Adjusted R^2: 0.9434, RMSE: 2.2184
Evaluating {'learning_rate': 0.05, 'n_estimators': 800, 'max_depth': 10, 'subsample': 0.6, 'colsample_bytree': 1, 'gamma': 0, 'alpha': 1.5, 'lambda': 1, 'colsample_bylevel': 0.6, 'colsample_bynode': 0.8} took 35.34 seconds
MSE: 4.8325, MAE: 1.0939, R^2: 0.9444, Adjusted R^2: 0.9444, RMSE: 2.1983
Evaluating {'learning_rate': 0.05, 'n_estimators': 800, 'max_depth': 10, 'subsample': 0.6, 'colsample_bytree'

In [107]:
results_df = pd.DataFrame(results)

In [109]:
print(results_df.head())

# Save DataFrame to CSV file in the same directory as the Jupyter Notebook
results_df.to_csv('results.csv', index=False)

                                          Parameters       MSE       MAE  \
0  {'learning_rate': 0.05, 'n_estimators': 800, '...  4.912828  1.114663   
1  {'learning_rate': 0.05, 'n_estimators': 800, '...  4.921252  1.111488   
2  {'learning_rate': 0.05, 'n_estimators': 800, '...  4.832467  1.093934   
3  {'learning_rate': 0.05, 'n_estimators': 800, '...  4.972566  1.115476   
4  {'learning_rate': 0.05, 'n_estimators': 800, '...  4.805282  1.092910   

        R^2  Adjusted R^2      RMSE   Duration  
0  0.943473      0.943454  2.216490  30.959794  
1  0.943376      0.943357  2.218389  37.214747  
2  0.944398      0.944379  2.198287  35.337413  
3  0.942786      0.942766  2.229925  40.090993  
4  0.944711      0.944692  2.192095  40.372811  
